In [11]:
# OpenAIの作成

from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.chunk_size = 512

In [12]:
# Nebula Graphのコンフィグ

import os

os.environ["NEBULA_USER"] = "root"
os.environ["NEBULA_PASSWORD"] = "nebula"  # default is "nebula"
os.environ["NEBULA_ADDRESS"] = (
    "127.0.0.1:9669"  # assumed we have NebulaGraph installed locally
)

space_name = "boxerengine"
edge_types, rel_prop_names = ["relationship"], [
    "relationship"
]  # default, could be omit if create from an empty kg
tags = ["entity"]  # default, could be omit if create from an empty kg

In [13]:
# NebulaGraphの接続
# 「llamaindexのスペースの作成手順」を実施後に実行する

from llama_index.core import StorageContext
from llama_index.graph_stores.nebula import NebulaGraphStore

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [15]:
# Documentのサンプル情報を取得する

from llama_index.readers.wikipedia import WikipediaReader

loader = WikipediaReader()
documents = loader.load_data(
    pages=["水平対向エンジン"], auto_suggest=False, lang_prefix="ja"
)

ConnectionError: HTTPConnectionPool(host='en.wikipedia.org', port=80): Max retries exceeded with url: /w/api.php?meta=siteinfo&siprop=languages&format=json&action=query (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7fce79c7a590>: Failed to resolve 'en.wikipedia.org' ([Errno -3] Temporary failure in name resolution)"))

In [ ]:
# KnowledgeGraphIndexを構築する

from llama_index.core import KnowledgeGraphIndex

kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    # space_name=space_name,
    # edge_types=edge_types,
    # rel_prop_names=rel_prop_names,
    # tags=tags,
    include_embeddings=True,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST

In [ ]:
# Nebula Graphに接続してSpaceの情報を取得する
%load_ext ngql
%ngql --address 127.0.0.1 --port 9669 --user root --password root

Connection Pool Created
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,Name
0,llamaindex


In [ ]:
# llamaindexの検索を行う

%ngql USE llamaindex;
%ngql MATCH ()-[e]->() RETURN e LIMIT 10

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,e
0,"(""Adam"")-[:relationship@9005598024634048093{relationship: ""Stabbed by""}]->(""Nebula"")"
1,"(""Adam mckay"")-[:relationship@-7875718538987686817{relationship: ""Said he was willing to work with""}]->(""Marvel studios again"")"
2,"(""Adam warlock"")-[:relationship@-7355033301802044942{relationship: ""Created by""}]->(""High priestess ayesha"")"
3,"(""Adam warlock"")-[:relationship@-6025549415605064876{relationship: ""Importance on""}]->(""Cosmic side"")"
4,"(""Adam warlock"")-[:relationship@4003220439395659920{relationship: ""Created by""}]->(""High priestess ayesha"")"
5,"(""Adam warlock"")-[:relationship@7673960981714070136{relationship: ""Noted""}]->(""Importance"")"
6,"(""Alan f. horn"")-[:relationship@2023067982055303897{relationship: ""Stated""}]->(""Offensive attitudes"")"
7,"(""Alan f. horn"")-[:relationship@7888911368531140309{relationship: ""Stated""}]->(""Offensive attitudes"")"
8,"(""Allmovie"")-[:relationship@-9163102535676806896{relationship: ""At""}]->(""Guardians of the galaxy vol. 3"")"
9,"(""Allmovie"")-[:relationship@2569255774608074042{relationship: ""At""}]->(""Guardians of the galaxy vol. 3"")"


In [ ]:
# draw the result
# 結果が出力されない
%ng_draw

<class 'pyvis.network.Network'> |N|=12 |E|=10